In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
from requests_futures.sessions import FuturesSession
from urllib.parse import urljoin
from collections import namedtuple
from selenium.webdriver.chrome.options import Options

In [3]:
SCROLL_PAUSE_TIME = 0.8
start = time.time()
keywords = ['data science', 'data analyst', 'data engineer', 'business analyst', 'business intelligence']
driver = webdriver.Chrome(r'C:\Users\Admin\Desktop\chromedriver.exe')
soups=[]

for i in range(len(keywords)):
    driver.get('https://www.internship.edu.vn')
    input_kw = driver.find_element_by_id('search-keyword')
    input_kw.clear()
    input_kw.send_keys(keywords[i])
    input_kw.send_keys(Keys.RETURN)

    soups.append([])

    while True:
        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")

        while True:
            see_more = driver.find_elements_by_xpath('//div[@style = "opacity: 1; display: none;"]')

            if len(see_more) == 0:
                    driver.find_elements_by_xpath('//div[@class = "loadmore-action"]')[0].click()
                    time.sleep(SCROLL_PAUSE_TIME)
            else:
                    break

        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height



    soups[-1].append(bs(driver.page_source  , 'lxml'))

        



TimeoutException: Message: timeout
  (Session info: chrome=71.0.3578.98)
  (Driver info: chromedriver=2.45.615291 (ec3682e3c9061c10f26ea9e5cdcf3c53f3f74387),platform=Windows NT 10.0.17134 x86_64)


In [38]:
area = soups[0][0].find_all('div', attrs = {'class':'jobs posts-loop '})
area[0].select('h2')[0].select('a')[0].text
date = area[0].find('span', attrs = {'class':'job-date'})


<span class="job-date">
<time class="entry-date" datetime="2016-03-30T00:00:00+00:00">
<i class="fa fa-calendar"></i>
<span>
30/03/2016
</span>
<span>
 - 15/04/2016
</span>
</time>
</span>

In [3]:
urls = []
jobs = []
comps = []
location = []
for soup in soups:
    urls.append([])
    jobs.append([])
    comps.append([])
    location.append([])
    for small_soup in soup:
        url = small_soup.find_all('div', attrs = {'class':'jobs posts-loop '})
        urls[-1] += [link.select('h2')[0].select('a')[0]['href'] for link in url]
        jobs[-1] += [link.select('h2')[0].select('a')[0].text]
        comp = small_soup.find_all('p', attrs = {'class':'namecom'})
        for post in comp:
            if post.select('a') != []:
                comps[-1].append(post.select('a')[0].text)
            else:
                comps[-1].append('0')
  
        loc = small_soup.find_all('p', attrs = {'class':'location'})
        location[-1] += [post.text for post in loc]


In [4]:
TechDetailed = namedtuple('TechDetailed', 'description, requirement')
def detail(response):
    soup = bs(response.text, 'lxml')
    requirement = ''
    description = ''
    if soup.find('div', attrs = {'class':'TitleJoblarge'}) != None:
        requirement = 'check'
        description = 'check'
    elif soup.find('div', attrs = {'class':'title320'}) != None:
        text = soup.find_all('div', attrs = {'class': 'content_w350'})
        for i in text[0].select('p')[1:-1]:
            description += i.text
        for i in text[1].select('li'):
            requirement += i.text
    elif soup.find_all('div', attrs = {'class':'top-job-info'}) != None:
        contents = soup.find_all('div', attrs = {'class':'content_fck'})
        for i in contents[0].select('li'):
            description += i.text
        for i in contents[1].select('li'):
            requirement += i.text

    else:
        requirement = 'test'
        description = 'test'
    
    return TechDetailed(description, requirement)

In [5]:
session = FuturesSession(max_workers=20)
futures = [[session.get(url) for url in link] for link in urls]

In [6]:
details = [[detail(post.result()) for post in future] for future in futures] 
end = time.time()

In [8]:
end - start

102.39605450630188